Passo a passo do estudo:

1) Obtemos os dados da base Vendas.xlsx;

2) Organizamos os dados usando o PANDAS;

3) Extraimos da base de dados 3 tabelas específicas

    -> Faturamento por loja;
    -> Quantidade  vendida por loja;
    -> |Ticket médio por loja;

4) Ordenamos os dados, do maior para o menor, para facilitar a visualização
    
    -> Coletar as informações que geramos
    -> Criar o e-mail com os dados da loja;
    -> Enviar o e-mail para a pessoa responsável por aquela loja
    -> Enviar o e-mail geral para Diretoria


In [3]:
import pandas as pd

df = pd.read_excel(r'/content/drive/MyDrive/Colab Notebooks/Projetos/Vendas.xlsx')
display(df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
100995,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
100996,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108


In [4]:
faturamento = df[['ID Loja', 'Valor Final']].groupby('ID Loja').sum()
faturamento = faturamento.sort_values(by='Valor Final', ascending = False)
display(faturamento)

,Valor Final
ID Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


In [7]:
quantidade = df[['ID Loja', 'Quantidade']].groupby('ID Loja').sum()
quantidade = quantidade.sort_values(by = 'ID Loja', ascending= False)
display(quantidade)

,Quantidade
ID Loja,
Shopping Vila Velha,9224
Shopping União de Osasco,8730
Shopping SP Market,8927
Shopping Recife,8581
Shopping Morumbi,8508
Shopping Midway Mall,8206
Shopping Iguatemi Fortaleza,8629
Shopping Ibirapuera,8723
Shopping Eldorado,8719


In [8]:
ticket_medio = (faturamento['Valor Final']/ quantidade ['Quantidade']).to_frame()
ticket_medio = ticket_medio.rename(columns={0: 'Ticket Medio'})

In [10]:
lojas = df['ID Loja'].unique()

for loja in lojas:
  tabela_loja = df.loc[df['ID Loja'] == loja, ['ID Loja', 'Quantidade', 'Valor Final']]
  resumo_loja = tabela_loja.groupby('ID Loja').sum()
  resumo_loja['Ticket Medio'] = resumo_loja['Valor Final'] / resumo_loja['Quantidade']

In [11]:
tabela_diretoria = faturamento.join(quantidade).join(ticket_medio)
display(tabela_diretoria)

,Valor Final,Quantidade,Ticket Medio
ID Loja,,,
Iguatemi Campinas,1762419,8935,197.248909
Shopping Vila Velha,1731167,9224,187.680724
Bourbon Shopping SP,1726110,8863,194.754598
Rio Mar Recife,1722766,8863,194.377299
Shopping SP Market,1721763,8927,192.871401
Palladium Shopping Curitiba,1721120,9091,189.321307
Norte Shopping,1711968,9014,189.923231
Ribeirão Shopping,1707122,8825,193.441586
Iguatemi Esplanada,1699681,8580,198.098019


In [ ]:
import smtplib
import email.message

def enviar_email(resumo_loja, loja):
  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = "Corpo do E-mail"
    
  msg = email.message.Message()
  msg['Subject'] = "Assunto do E-mail"
    
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
    # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = 'remetente@gmail.com'
  msg['To'] = 'destinatario@gmail.com'
  password = "senha"
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')